In [3]:
import torch
import torch.nn.functional as F
import numpy as np

In [4]:
input = np.array([[2.0,1.0,0.1]])

In [5]:
tempreture = 1
logits = torch.tensor(input /tempreture)
softmax_score = F.softmax(logits, dim=1)
print(f"temperature = {tempreture} {softmax_score.numpy()}")

temperature = 1 [[0.65900114 0.24243297 0.09856589]]


In [6]:
tempreture =0.1
logits = torch.tensor(input /tempreture)
softmax_score = F.softmax(logits, dim=1)
print(f"temperature = {tempreture} {softmax_score.numpy()}")

temperature = 0.1 [[9.99954597e-01 4.53978684e-05 5.60254205e-09]]


In [7]:
tempreture = 20
logits = torch.tensor(input /tempreture)
softmax_score = F.softmax(logits, dim=1)
print(f"temperature = {tempreture} {softmax_score.numpy()}")

temperature = 20 [[0.34957672 0.33252767 0.31789561]]


In [8]:
# 调用大模型的集中方法
# 1. modelscope
from modelscope import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [11]:
model_path = '../model/Qwen2.5-0.5B-Instruct/'
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)
gen_config = GenerationConfig.from_pretrained(model_path)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [12]:
tokenizer("你好，are you OK?")

{'input_ids': [108386, 3837, 546, 498, 10402, 30], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [13]:
tokenizer.decode(108386)

'你好'

In [14]:
gen_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

In [75]:
def run_prompt(prompt, temperature =0.1, top_k=20, top_p=0.8, max_new_tokens=2048):
    gen_config.temperature = temperature
    gen_config.top_k =top_k
    gen_config.top_p = top_p

    messages = [
        {"role":"system","content":""},
        {"role":"user","content":prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    print(text)
    print("----------------")
    model_input = tokenizer([text], return_tensors="pt")
    generated_ids = model.generate(model_input.input_ids, max_new_tokens = max_new_tokens,generation_config=gen_config)
    print("model_inputids:",model_input.input_ids)
    print("generated_ids:", generated_ids)
    response_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_input.input_ids, generated_ids)]
    print("response_ids:",response_ids)
    response = tokenizer.batch_decode(response_ids, skip_special_tokens=True)[0]
    return response

prompt = "Hello"

run_prompt(prompt)

<|im_start|>system
<|im_end|>
<|im_start|>user
Hello<|im_end|>
<|im_start|>assistant

----------------
model_inputids: tensor([[151644,   8948,    198, 151645,    198, 151644,    872,    198,   9707,
         151645,    198, 151644,  77091,    198]])
generated_ids: tensor([[151644,   8948,    198, 151645,    198, 151644,    872,    198,   9707,
         151645,    198, 151644,  77091,    198,   9707,      0,   2585,    646,
            358,   7789,    498,   3351,     30, 151645]])
response_ids: [tensor([  9707,      0,   2585,    646,    358,   7789,    498,   3351,     30,
        151645])]


'Hello! How can I assist you today?'

In [32]:
tokenizer.decode(10)

'吗'

In [42]:
in_ids = torch.tensor([[151644,   8948,    198, 151645,    198, 151644,    872,    198, 108386,
         151645,    198, 151644,  77091,    198]])
gen_ids = torch.tensor([[151644,   8948,    198, 151645,    198, 151644,    872,    198, 108386,
         151645,    198, 151644,  77091,    198, 108386,   6313, 104139, 111728,
         103929, 101037,  11319, 151645]])

In [46]:
print(in_ids.shape)
print(gen_ids.shape)

torch.Size([1, 14])
torch.Size([1, 22])


In [65]:
for i, j in zip(in_ids, gen_ids):
    print("i=",i)
    print(i.shape)
    print(len(i))
    print("----------------------")
    print("j=",j)
    print(j.shape)
    z = j[11:]
    print(z)
   


i= tensor([151644,   8948,    198, 151645,    198, 151644,    872,    198, 108386,
        151645,    198, 151644,  77091,    198])
torch.Size([14])
14
----------------------
j= tensor([151644,   8948,    198, 151645,    198, 151644,    872,    198, 108386,
        151645,    198, 151644,  77091,    198, 108386,   6313, 104139, 111728,
        103929, 101037,  11319, 151645])
torch.Size([22])
tensor([151644,  77091,    198, 108386,   6313, 104139, 111728, 103929, 101037,
         11319, 151645])


In [85]:
# 2通过transformer调用模型
from transformers import AutoModel, AutoTokenizer
model_path = '../model/Qwen1.5-0.5B-Chat/'
tokenizer= AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True)

In [87]:
print(model)

Qwen2Model(
  (embed_tokens): Embedding(151936, 1024)
  (layers): ModuleList(
    (0-23): 24 x Qwen2DecoderLayer(
      (self_attn): Qwen2Attention(
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
      )
      (mlp): Qwen2MLP(
        (gate_proj): Linear(in_features=1024, out_features=2816, bias=False)
        (up_proj): Linear(in_features=1024, out_features=2816, bias=False)
        (down_proj): Linear(in_features=2816, out_features=1024, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): Qwen2RMSNorm((1024,), eps=1e-06)
      (post_attention_layernorm): Qwen2RMSNorm((1024,), eps=1e-06)
    )
  )
  (norm): Qwen2RMSNorm((1024,), eps=1e-06)
  (rotary_emb): Qwen2RotaryEmbedding()
)


In [88]:
def chatglm(prompt, history=[], model = model, tokenizer=tokenizer):
    response, history = model.generate(tokenizer,
                                  prompt,
                                  history=history,
                                  tempreture=0.1,
                                  top_p=0.8,
                                  top_k=20,
                                  max_length=8192)
    return response

chatglm("你好")



TypeError: The current model class (Qwen2Model) is not compatible with `.generate()`, as it doesn't have a language model head. Classes that support generation often end in one of these names: ['ForCausalLM', 'ForConditionalGeneration', 'ForSpeechSeq2Seq', 'ForVision2Seq'].

In [108]:
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="../model/Qwen1.5-0.5B-Chat")
response = pipe(messages)
response[0]['generated_text'][1]['content']

Device set to use cpu


'I am a large language model created by Alibaba Cloud. I am called Qwen.'

In [110]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "../model//Qwen1.5-0.5B-Chat",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("../model//Qwen1.5-0.5B-Chat")

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt")

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


In [111]:
print(response)



A large language model is a type of machine learning algorithm that can generate human-like text based on a given input, often in the form of a prompt or question. These models can be used for various purposes, such as language translation, text generation, sentiment analysis, and more. They have become increasingly important in recent years due to their ability to handle large amounts of data and quickly learn patterns and relationships within that data.


In [112]:
from openai import OpenAI

In [124]:
client = OpenAI(
    api_key = 'xx',
    base_url='http://localhost:11434/v1'
)
reponse = client.chat.completions.create(
    model = 'qwen2.5:3b',
    messages = [
        {"role":"system","content":""},
        {"role":"user","content":"你能做什么"}
    ],
    temperature = 0.7,
    stream = True
)
print (reponse)

In [126]:
for chunk in reponse:
    print(chunk.choices[0].delta.content, end='', flush=True)
    print(chunk)